#  Submit  NEB or Replica chain

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# General imports.
import ipywidgets as ipw
from IPython.display import clear_output

# AiiDA imports.
%load_ext aiida
%aiida
from aiida.plugins import  WorkflowFactory , DataFactory
from aiida.orm import  load_code

# AiiDAlab imports.
import aiidalab_widgets_base as awb

# Custom imports.
from widgets.build_slab import BuildSlab
from widgets.computational_resources import ProcessResourcesWidget, ResourcesEstimatorWidget
from widgets.inputs import InputDetails
from widgets.empa_viewer import EmpaStructureViewer
from widgets import utils
from widgets.ANALYZE_structure import StructureAnalyzer
#from widgets.import_cdxml import CdxmlUpload2GnrWidget



In [ ]:
Cp2kNebWorkChain = WorkflowFactory('nanotech_empa.cp2k.neb')
Cp2kReplicaWorkChain = WorkflowFactory('nanotech_empa.cp2k.replica')

In [ ]:
# Structure selector.
empa_viewer = EmpaStructureViewer()
build_slab = BuildSlab(title='Build slab')
ipw.dlink((empa_viewer, 'details'), (build_slab, 'details'))
ipw.dlink((empa_viewer, 'structure'), (build_slab, 'molecule'))

structure_selector = awb.StructureManagerWidget(
    viewer=empa_viewer,
    importers=[
        awb.StructureUploadWidget(title="Import from computer"),
        awb.StructureBrowserWidget(title="AiiDA database"),
        awb.OptimadeQueryWidget(embedded=True),
        awb.SmilesWidget(title="From SMILES"),
        #CdxmlUpload2GnrWidget(title="CDXML"),
    ],
    editors = [
        awb.BasicStructureEditor(title="Edit structure"),
        build_slab
    ],
    storable=False, node_class='StructureData')
display(structure_selector)

# Code.
computational_resources = awb.ComputationalResourcesWidget(
            description="CP2K code:", default_calc_job_plugin="cp2k"
        )
resources = ProcessResourcesWidget()

input_details = InputDetails()
replica_or_neb = ipw.ToggleButtons(options=['Replica', 'NEB'], 
                                   description='Type:', 
                                   disabled=False, 
                                   button_style='', 
                                   tooltips=['Replica', 'NEB'], 
                                   icons=['check', 'check'], 
                                   style={'description_width': 'initial'},
                                   value="NEB")
def on_replica_or_neb_change(change):
    global label, the_workchain
    if change['new'] == 'Replica':
        label = "CP2K_Replica"
        the_workchain = Cp2kReplicaWorkChain
        input_details.replica = True
        input_details.neb = False
    else:
        label = "CP2K_NEB"
        the_workchain = Cp2kNebWorkChain
        input_details.replica = False
        input_details.neb = True

replica_or_neb.observe(on_replica_or_neb_change, names='value')
label = "Cp2kNebWorkChain"
the_workchain = Cp2kNebWorkChain
input_details.replica= False
input_details.neb = True
create_input = ipw.Button(description="Create Input")

# Resources

In [ ]:
ipw.dlink((empa_viewer, 'details'), (input_details, 'details'))
ipw.dlink((computational_resources, 'value'),(input_details, 'selected_code'))

def prepare_inputs():
    parameters = input_details.final_dictionary.copy()

        self.ctx.wfn_cp_commands = mk_wfn_cp_commands(
            self.ctx.neb_params["number_of_replica"],
            uuids_to_check,
            self.inputs.code.computer,
        )    

    builder = the_workchain.get_builder()
    builder.metadata.label = label
    builder.metadata.description = parameters['description']    
    builder.code = load_code(computational_resources.value)
    builder.options = {
            "max_wallclock_seconds": resources.walltime_seconds,
            "resources": {
                'num_machines': resources.nodes,
                'num_mpiprocs_per_machine': resources.tasks_per_node,
                'num_cores_per_mpiproc': resources.threads_per_task,
            },
        }
    
    builder.structure = structure_selector.structure_node
    builder.dft_params = Dict(parameters['dft_params'])
    builder.sys_params = Dict(parameters['sys_params'])
    if 'neb_params' in parameters:
        builder.neb_params = Dict(parameters['neb_params'])
    
    if 'replica_uuids' in parameters:
        builder.replica_uuids = List(parameters['replica_uuids'])
    if 'restart_from' in parameters:
        builder.restart_from = Str(parameters['restart_from'])
   
    return builder

In [ ]:
def after_submission(_=None):   
    structure_selector.value = None
    
btn_submit_neb = awb.SubmitButtonWidget(Cp2kNebWorkChain, 
                                inputs_generator=prepare_inputs
                               )
btn_submit_replica = awb.SubmitButtonWidget(Cp2kReplicaWorkChain,
                                inputs_generator=prepare_inputs
                                 )

btn_submit_neb.on_submitted(after_submission)
btn_submit_replica.on_submitted(after_submission)

In [ ]:
output = ipw.Output()
def update_all(_=None):
    btn_submit_neb.btn_submit.disabled=True
    btn_submit_replica.btn_submit.disabled=True
 

def create(_=None):
    if computational_resources.value is None:
        msg = 'Please selecte a code first.'
        can_submit = False
    else:            
        can_submit, msg, details_dict = input_details.return_final_dictionary()
 
    with output:
        clear_output()
        if can_submit:
            print("Ready to submit a ",details_dict['system_type'], label, " calculation." )
            btn_submit_neb.btn_submit.disabled = not can_submit
            btn_submit_replica.btn_submit.disabled = not can_submit
            if replica_or_neb.value == 'Replica':
                display(btn_submit_replica)
            else:
                display(btn_submit_neb)
        else:
            print(msg)

create_input.on_click(create)
input_details.observe(update_all, names='details')

In [ ]:
# Resources estimation.
resources_estimation = ResourcesEstimatorWidget()
resources_estimation.link_to_resources_widget(resources)
ipw.dlink((structure_selector, 'structure'), (input_details, 'ase_atoms'))
ipw.dlink((empa_viewer, 'details'), (resources_estimation, 'details'))
ipw.dlink((input_details, 'uks'), (resources_estimation, 'uks'))
ipw.dlink((input_details, 'neb'),(resources, 'neb'))
ipw.dlink((input_details, 'n_replica_trait'),(resources, 'n_replica_trait'))
ipw.dlink((input_details, 'n_replica_per_group_trait'),(resources, 'n_replica_per_group_trait'))
ipw.dlink((resources, 'nproc_replica_trait'),(input_details, 'nproc_replica_trait'))
_ = ipw.dlink((computational_resources, 'value'), (resources_estimation, 'selected_code'))

In [ ]:
display(ipw.VBox([replica_or_neb,
                  input_details, 
                  resources, 
                  computational_resources,
                  resources_estimation,  
                  create_input]), 
        output)